DataSet: Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother, who lived in the woods. One day, her mother asked her to take a basket of goodies to her grandmother. On her way through the woods, she met a big bad wolf who wanted to eat her.

1. Build the Transformer Model on above dataset

In [1]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

# Load pre-trained model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Example text to train on
text = "Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother, who lived in the woods. One day, her mother asked her to take a basket of goodies to her grandmother. On her way through the woods, she met a big bad wolf who wanted to eat her."

# Tokenize input text
inputs = tokenizer(text, return_tensors="pt")

# Forward pass to train the model (requires more data for a full training loop)
outputs = model(**inputs, labels=inputs["input_ids"])
loss = outputs.loss
print(f"Loss: {loss.item()}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Loss: 2.5232996940612793


2. Train the model using 20, 60, 70 epochs

In [3]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
from torch.utils.data import DataLoader, Dataset

# Define a simple custom dataset
class SimpleTextDataset(Dataset):
    def __init__(self, text, tokenizer, max_length=128):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.encodings = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors="pt"
        )

    def __len__(self):
        return len(self.encodings.input_ids)

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings.input_ids[idx],
            "attention_mask": self.encodings.attention_mask[idx],
            "labels": self.encodings.input_ids[idx]
        }

# Load the pre-trained model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Add a padding token if it doesn't exist
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))  # Resize model's embedding layer to match the new tokenizer

# Example text
text = "Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother, who lived in the woods. One day, her mother asked her to take a basket of goodies to her grandmother. On her way through the woods, she met a big bad wolf who wanted to eat her."

# Prepare the dataset and data loader
dataset = SimpleTextDataset(text, tokenizer)
data_loader = DataLoader(dataset, batch_size=1, shuffle=True)

# Define optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# Training function
def train_model(epochs):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in data_loader:
            optimizer.zero_grad()  # Clear gradients

            # Move data to GPU if available
            input_ids = batch["input_ids"].to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
            attention_mask = batch["attention_mask"].to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
            labels = batch["labels"].to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

            # Forward pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()  # Backpropagation
            optimizer.step()  # Update weights

            total_loss += loss.item()

        print(f"Epoch {epoch + 1}/{epochs} - Loss: {total_loss / len(data_loader)}")

# Train the model for 20, 60, and 70 epochs
print("Training for 20 epochs:")
train_model(20)

print("\nTraining for 60 epochs:")
train_model(60)

print("\nTraining for 70 epochs:")
train_model(70)


Training for 20 epochs:
Epoch 1/20 - Loss: 46.73530578613281
Epoch 2/20 - Loss: 35.8935432434082
Epoch 3/20 - Loss: 25.874677658081055
Epoch 4/20 - Loss: 21.907794952392578
Epoch 5/20 - Loss: 12.615822792053223
Epoch 6/20 - Loss: 7.778767108917236
Epoch 7/20 - Loss: 5.7214460372924805
Epoch 8/20 - Loss: 4.422235012054443
Epoch 9/20 - Loss: 2.691516160964966
Epoch 10/20 - Loss: 5.495143890380859
Epoch 11/20 - Loss: 2.3937482833862305
Epoch 12/20 - Loss: 2.6641502380371094
Epoch 13/20 - Loss: 1.97486412525177
Epoch 14/20 - Loss: 3.0166516304016113
Epoch 15/20 - Loss: 3.5068209171295166
Epoch 16/20 - Loss: 2.3823318481445312
Epoch 17/20 - Loss: 2.4827966690063477
Epoch 18/20 - Loss: 3.017263889312744
Epoch 19/20 - Loss: 1.998845100402832
Epoch 20/20 - Loss: 2.536600351333618

Training for 60 epochs:
Epoch 1/60 - Loss: 1.8520265817642212
Epoch 2/60 - Loss: 1.9499835968017578
Epoch 3/60 - Loss: 2.6701929569244385
Epoch 4/60 - Loss: 1.7876782417297363
Epoch 5/60 - Loss: 1.768882155418396
Epo

3. After training, use the model to generate new text by feeding it an initial seed text

In [4]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load the tokenizer and the trained model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Ensure that the model's embedding layer is resized to include the new padding token
model.resize_token_embeddings(len(tokenizer))

# Move the model to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()  # Set the model to evaluation mode

# Seed text for generation
seed_text = "Once upon a time in a magical forest,"

# Encode the seed text
input_ids = tokenizer.encode(seed_text, return_tensors="pt").to(device)

# Generate new text
output = model.generate(
    input_ids,
    max_length=100,  # Length of the generated text
    num_return_sequences=1,  # Number of sequences to generate
    temperature=1.0,  # Control randomness: lower values make output more predictable
    top_p=0.9,  # Top-p sampling (nucleus sampling)
    do_sample=True  # Use sampling instead of greedy decoding
)

# Decode the generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("Generated Text:")
print(generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Generated Text:
Once upon a time in a magical forest, a small group of men and women made their way through the woods, slowly but surely gathering together, fighting their way through hordes of beasts and giants to reach the end.

At the end of that night, the last of their number went to an unknown location on the other side of the forest. There they began a new phase.

"Hahaha, it feels like the rest of us are going to stay here."

"


4. Experimenting and Improving the Model by large dataset and hyper tune parameter.

In [5]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
import torch
from torch.utils.data import DataLoader, Dataset
import os

# Define a custom dataset for larger data
class LargeTextDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=128):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.encodings = tokenizer(
            texts,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt"
        )

    def __len__(self):
        return len(self.encodings.input_ids)

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings.input_ids[idx],
            "attention_mask": self.encodings.attention_mask[idx],
            "labels": self.encodings.input_ids[idx]
        }

# Load the pre-trained model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))  # Resize for the new padding token

# Load and preprocess your large dataset
texts = [
    # Add your large collection of text data here, such as a list of fairy tales or story paragraphs
    "Once upon a time...",
    "Another example sentence...",
    "... more text ..."
]

dataset = LargeTextDataset(texts, tokenizer, max_length=256)  # Increase max_length for longer texts
data_loader = DataLoader(dataset, batch_size=8, shuffle=True)  # Adjust batch size as needed

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define the optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=3e-5)  # Adjust learning rate as needed
num_epochs = 5
total_steps = len(data_loader) * num_epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,  # Number of warmup steps
    num_training_steps=total_steps
)

# Training loop with hyperparameter tuning
def train_model(epochs):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in data_loader:
            optimizer.zero_grad()  # Clear gradients

            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()  # Backpropagation
            optimizer.step()  # Update weights
            scheduler.step()  # Update learning rate

            total_loss += loss.item()

        print(f"Epoch {epoch + 1}/{epochs} - Loss: {total_loss / len(data_loader)}")

# Train the model
train_model(num_epochs)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/5 - Loss: 1.0138740539550781
Epoch 2/5 - Loss: 0.8844705820083618
Epoch 3/5 - Loss: 0.8709707856178284
Epoch 4/5 - Loss: 0.7695357799530029
Epoch 5/5 - Loss: 0.6661518812179565
